In [ ]:
## Github
## audio folder: https://github.com/ming024/FastSpeech2/blob/master/audio
##  ㄴ stft.py : https://github.com/ming024/FastSpeech2/blob/master/audio/stft.py
##  ㄴ audio_processing: https://github.com/ming024/FastSpeech2/blob/master/audio/audio_processing.py
##  ㄴ tools.py: https://github.com/ming024/FastSpeech2/blob/master/audio/tools.py

# Configuration

In [5]:
#### config/LibriTTS/preprocess.yaml: https://github.com/ming024/FastSpeech2/blob/master/config/LibriTTS/preprocess.yaml

# sampling_rate = 22050
sample_rate = 22050
max_wav_value = 32768.0

### stft
filter_length = 1024
n_fft =  1024 ## maybe filter_length? 

hop_length = 256
win_length = 1024
## mel
n_mel_channels = 80

#   audio:
#     sampling_rate: 22050
#     max_wav_value: 32768.0
#   stft:
#     filter_length: 1024
#     hop_length: 256
#     win_length: 1024
#   mel:
#     n_mel_channels: 80
#     mel_fmin: 0
#     mel_fmax: 8000 # please set to 8000 for HiFi-GAN vocoder, set to null for MelGAN vocoder
#   pitch:
#     feature: "phoneme_level" # support 'phoneme_level' or 'frame_level'
#     normalization: True
#   energy:
#     feature: "phoneme_level" # support 'phoneme_level' or 'frame_level'
#     normalization: True

# import

In [19]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import librosa
import librosa.display
import librosa.util as librosa_util
from scipy.signal import get_window

import torch
import torch.nn as nn

import IPython.display as ipd

In [2]:
mini_data_path = '/home/heiscold/prac/train_part_20k.csv'

df = pd.read_csv('/home/heiscold/prac/train_part_20k.csv')
print(df.shape)
df.head()

(20000, 6)


,file_id,speaker_id,chapter_id,utterance_id,sentence,audio_path
0,100_2315_000000,100,2315,0,come let's go said trot again so they left the...,/data/speech-data/mls/mls_english_opus/train/a...
1,100_2315_000001,100,2315,1,indeed the sea devils had crept upon them and ...,/data/speech-data/mls/mls_english_opus/train/a...
2,100_2315_000002,100,2315,2,once in a while one of those crowded out of pl...,/data/speech-data/mls/mls_english_opus/train/a...
3,100_2315_000003,100,2315,3,sacho's thin little legs trotted through the w...,/data/speech-data/mls/mls_english_opus/train/a...
4,100_2315_000004,100,2315,4,here reclined queen aquareine a being so lovel...,/data/speech-data/mls/mls_english_opus/train/a...


## Sample

In [7]:
idx = 12345
audio_path = df.audio_path.values[idx]
sentence = df.sentence.values[idx]

print(f"Audio Path: {audio_path}")
print(f"SENTENCE: {sentence}")


audio, sr  = librosa.load(audio_path, sr=None)
# audio, sr  = torchaudio.load(audio_path,)
# returned sr is 'orig_sr' of sound
print("ORIGINAL SAMPLE RATE: ", sr)
print("TARGET SAMPLE RATE", sample_rate)

## SAMPLE
audio = librosa.resample(audio, orig_sr= sr, target_sr= sample_rate )
# resample = torchaudio.transforms.Resample(orig_freq=sr, new_freq=sample_rate,)
# audio = resample(audio)
ipd.display(ipd.Audio(audio, rate = sample_rate))

Audio Path: /data/speech-data/mls/mls_english_opus/train/audio/10018/12447/10018_12447_004110.opus
SENTENCE: you have news i asked as i took her hand i have the kind of news i expected she answered a smile tempering the gravity of her face auguste is as usual in need of money
ORIGINAL SAMPLE RATE:  16000
TARGET SAMPLE RATE 22050


# STFT Class 
## `def __init__` 

In [18]:
# np.eye(5), (np.fft.fft(np.eye(5))), (np.fft.fft(np.eye(5))).shape

In [17]:
fourier_basis = np.fft.fft(np.eye(filter_length)) ## 대각 행렬 
print(fourier_basis.shape) ## (filter_length, filter_length)

# cutoff = int((self.filter_length / 2 + 1))
# fourier_basis = np.vstack(
#     [np.real(fourier_basis[:cutoff, :]), np.imag(fourier_basis[:cutoff, :])]
# )
cutoff = int(filter_length /2  + 1) # 513
fourier_basis = np.vstack([np.real(fourier_basis[:cutoff, :]), ## 실수부 # [filter_length //2 + 1, filter_length]
                           np.imag(fourier_basis[:cutoff, :])  ## 허수부 # [filter_length //2 + 1, filter_length]
                           ] )
## fourier_basis Shape: # (filter_length, filter_length)
cutoff, fourier_basis.shape, # fourier_basis

(1024, 1024)


(513, (1026, 1024))

In [25]:
# forward_basis = torch.FloatTensor(fourier_basis[:, None, :])

forward_basis = torch.FloatTensor(fourier_basis[:, None, :]) # torch tensor & [filter_lengt + 2, 1, filter_length]
## why filter_length + 2 ?????
##  ---> cutoff(=int(filter_length //2  + 1)) * 2

scale = filter_length / hop_length ## 겹치는 부분인가? 뭐지...?
inverse_basis = torch.FloatTensor(
    np.linalg.pinv(scale * fourier_basis).T[:, None, :]
    # scale * fourier_basis : [filter_lengt + 2, 1, filter_length]
    # np.linalg.pinv: 역행렬 구하는 함수 // Shape: [filter_length, filter_length + 2]
    # np.linalg.pinv(scale * fourier_basis).T[:, None, :]  ---> 이전 Shape으로 돌아감: [filter_lengt + 2, 1, filter_length] (=1026, 1, 1024)
)

forward_basis.shape, inverse_basis.shape

(torch.Size([1026, 1, 1024]), torch.Size([1026, 1, 1024]))

In [29]:
from librosa.util import pad_center, tiny

#  if window is not None:
#     assert filter_length >= win_length
#      get window and zero center pad it to filter_length

## scipy -> get_window: https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.get_window.html
window = "hann"
fft_window = get_window(window, win_length, fftbins=True)
fft_window.shape, fft_window 

((1024,),
 array([0.00000000e+00, 9.41235870e-06, 3.76490804e-05, ...,
        8.47091021e-05, 3.76490804e-05, 9.41235870e-06]))

In [34]:
fft_window = get_window(window, win_length, fftbins=True)
fft_window = pad_center(data =fft_window, size = filter_length) ## Check
fft_window.shape

(1024,)

In [35]:
fft_window = torch.from_numpy(fft_window).float()
fft_window.shape

torch.Size([1024])

In [36]:
# window the bases

# forward_basis: [1026, 1, 1024]
print(forward_basis)
forward_basis *= fft_window
print(forward_basis.shape) # [1026, 1, 1024]

tensor([[[ 1.0000,  1.0000,  1.0000,  ...,  1.0000,  1.0000,  1.0000]],

        [[ 1.0000,  1.0000,  0.9999,  ...,  0.9998,  0.9999,  1.0000]],

        [[ 1.0000,  0.9999,  0.9997,  ...,  0.9993,  0.9997,  0.9999]],

        ...,

        [[ 0.0000, -0.0123,  0.0245,  ...,  0.0368, -0.0245,  0.0123]],

        [[ 0.0000, -0.0061,  0.0123,  ...,  0.0184, -0.0123,  0.0061]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])
torch.Size([1026, 1, 1024])


In [38]:
# forward_basis *= fft_window
forward_basis[:5]

tensor([[[0.0000e+00, 9.4124e-06, 3.7649e-05,  ..., 8.4709e-05,
          3.7649e-05, 9.4124e-06]],

        [[0.0000e+00, 9.4122e-06, 3.7646e-05,  ..., 8.4695e-05,
          3.7646e-05, 9.4122e-06]],

        [[0.0000e+00, 9.4117e-06, 3.7638e-05,  ..., 8.4652e-05,
          3.7638e-05, 9.4117e-06]],

        [[0.0000e+00, 9.4108e-06, 3.7624e-05,  ..., 8.4580e-05,
          3.7624e-05, 9.4108e-06]],

        [[0.0000e+00, 9.4095e-06, 3.7604e-05,  ..., 8.4480e-05,
          3.7604e-05, 9.4095e-06]]])

In [39]:
inverse_basis *= fft_window
print(inverse_basis.shape)

torch.Size([1026, 1, 1024])


##  `def transform(self, input_data):`

In [40]:
#   def transform(self, input_data):

input_data = torch.tensor(audio).reshape(1, -1)
print(input_data.shape)
# [추정] input_data: audio (= librosa.load or '-> resample') -> torch.tensor
# Spectrogram: [Batch Size, frame_legnth(=n_fft/2 + 1), N_Frames(=Audio_Length // hop_length)]
num_batches = 1 # input_data.size(0)
num_samples = input_data.size(1)
num_batches, num_samples

torch.Size([1, 325458])


(1, 325458)

In [41]:
# similar to librosa, reflect-pad the input
input_data = input_data.view(num_batches, 1, num_samples) # [num_batches, 1, num_samples]
print(input_data.shape)

torch.Size([1, 1, 325458])


In [45]:
import torch.nn.functional as F
## https://pytorch.org/docs/stable/generated/torch.nn.functional.pad.html


## Example from: https://freshrimpsushi.github.io/posts/hot-to-pad-a-tensor-in-pytorch/
B = torch.tensor([[[1,2,3,4,5.]],[[6,7,8,9,10]]])
# tensor([[[ 1.,  2.,  3.,  4.,  5.]],
#         [[ 6.,  7.,  8.,  9., 10.]]])
print(B.shape)
print(F.pad(B, (2,3), 'reflect').shape)
print(F.pad(B, (2,3), 'reflect'))
# tensor([[[ 3.,  2.,  1.,  2.,  3.,  4.,  5.,  4.,  3.,  2.]],
#         [[ 8.,  7.,  6.,  7.,  8.,  9., 10.,  9.,  8.,  7.]]])




torch.Size([2, 1, 5])
torch.Size([2, 1, 10])


tensor([[[ 3.,  2.,  1.,  2.,  3.,  4.,  5.,  4.,  3.,  2.]],

        [[ 8.,  7.,  6.,  7.,  8.,  9., 10.,  9.,  8.,  7.]]])

In [47]:
input_data = F.pad( input_data.unsqueeze(1), ## [num_batches, 1, num_samples] --> [num_batches, 1, 1, num_samples]
                    (int(filter_length / 2), int(filter_length / 2), 0, 0), ## (512, 512, 0, 0)
                    mode="reflect", ## 뭔가 반사틱하게 (=좌우 대칭틱하게 패딩...)
                    )
print(input_data.shape)

torch.Size([1, 1, 1, 326482])


In [48]:
input_data = input_data.squeeze(1) ## [num_batches, 1, 1, num_samples] --> [num_batches, 1, num_samples]
print(input_data.shape)

torch.Size([1, 1, 326482])


In [52]:
input_data[0, 0, 510: 515] ##

tensor([0.0007, 0.0008, 0.0007, 0.0008, 0.0007])

In [53]:
#  input_data.shape, forward_basis.shape
# (torch.Size([1, 1, 326482]), torch.Size([1026, 1, 1024]))



(torch.Size([1, 1, 326482]), torch.Size([1026, 1, 1024]))

In [56]:
forward_transform = F.conv1d(
            # input_data.cuda(),
            input_data, # Shape: [1, 1, 326482])

            # torch.autograd.Variable(self.forward_basis, requires_grad=False).cuda(),
            torch.autograd.Variable(forward_basis, requires_grad=False),
            # forward_basis - Shape: [1026, 1, 1024])

            stride= hop_length, padding=0,)

forward_transform.shape
## torch.Size([1, 1026, 1272])

torch.Size([1, 1026, 1272])

In [61]:
cutoff = int((filter_length / 2) + 1) ## 513 
real_part = forward_transform[:, :cutoff, :] ### 실수부...? ## [1, 513, 1272]
imag_part = forward_transform[:, cutoff:, :] ### 허수부...? ## [1, 513, 1272]
magnitude = torch.sqrt(real_part ** 2 + imag_part ** 2) ## Energy?

print(cutoff, real_part.shape, imag_part.shape, magnitude.shape )
phase = torch.autograd.Variable(torch.atan2(imag_part.data, real_part.data)) 
## Phase: 위상
## torch.atan2 :
####  atan은 두 점 사이의 탄젠트값을 받아 절대각을 -π/2 ~ π/2의 라디안 값으로 반환한다. (-90 ~ 90도)
####  atan2는 두 점 사이의 상대좌표(x, y)를 받아 절대각을 -π ~ π의 라디안 값으로 반환한다. (-180 ~ 180도)
phase.shape ## [1, 513, 1272]

513 torch.Size([1, 513, 1272]) torch.Size([1, 513, 1272]) torch.Size([1, 513, 1272])


torch.Size([1, 513, 1272])

##  `def inverse(self, magnitude, phase):`

In [62]:
# def inverse(self, magnitude, phase):
# magnitude ## from self.transform ## Shape: [1, 513, 1272]
# phase     ## from self.transform ## Shape: [1, 513, 1272]

recombine_magnitude_phase = torch.cat([magnitude * torch.cos(phase), magnitude * torch.sin(phase)], dim=1) ## Shape: [1, 1026, 1272]
## magnitude * cosine(phase) 
# ## magnitude * sin(phase)
recombine_magnitude_phase.shape

torch.Size([1, 1026, 1272])

In [65]:
inverse_transform = F.conv_transpose1d(
    recombine_magnitude_phase, ## [1, 1026, 1272]
    torch.autograd.Variable(inverse_basis, requires_grad=False), ## [1026, 1, 1024]
    stride=hop_length,
    padding=0,
)
print(audio.shape) # (325458,)
inverse_transform .shape # 1, 1, 326400

(325458,)


torch.Size([1, 1, 326400])

### window is not none

In [66]:
# if self.window is not None:
print(window)

hann


#### win_sumsquare()

In [83]:


def window_sumsquare(
    window,
    n_frames,
    hop_length,
    win_length,
    n_fft,
    dtype=np.float32,
    norm=None,
):
    if win_length is None:
        win_length = n_fft

    n = n_fft + hop_length * (n_frames - 1)
    x = np.zeros(n, dtype=dtype)

    # Compute the squared window at the desired length
    win_sq = get_window(window, win_length, fftbins=True)
    win_sq = librosa_util.normalize(win_sq, norm=norm) ** 2
    win_sq = librosa_util.pad_center(data=win_sq, size =n_fft)

    # Fill the envelope
    for i in range(n_frames):
        sample = i * hop_length
        x[sample : min(n, sample + n_fft)] += win_sq[: max(0, min(n_fft, n - sample))]
    return x

In [78]:
# def window_sumsquare(
#     window,
#     n_frames = magnitude.size(-1) # 1272 
#     hop_length,
#     win_length,
#     n_fft = filter_length
#     dtype=np.float32,
#     norm=None,
# ):
    # if win_length is None:
    #     win_length = n_fft ## filter_length
n_frames =  magnitude.size(-1) # 1272 
n_fft = filter_length
dtype=np.float32
norm = None
print(n_fft, filter_length, win_length, n_frames)
# 1024 1024 1024 1272

n = n_fft + hop_length * (n_frames - 1) ## n : audio length? aprrocimate?
x = np.zeros(n, dtype=dtype)            ## np.zeros (Length of n)
print(n, x.shape)
# 326400 (326400,)

# # Compute the squared window at the desired length
win_sq = get_window(window, win_length, fftbins=True) ## window = hann ## (1024)
win_sq = librosa_util.normalize(win_sq, norm=norm) ** 2 ## Energy??? Power? ## (1024)
# print(win_sq.shape) ## (1024)
win_sq = librosa_util.pad_center(data =win_sq, size = n_fft)
# print(win_sq.shape) # (1024)

# Fill the envelope
for i in range(n_frames):
    sample = i * hop_length # 1024, 2*1024, 3*1024, 4*1024, ...
    x[sample : min(n, sample + n_fft)] += win_sq[: max(0, min(n_fft, n - sample))]
    # min(n, sample + n_fft) --> sample + n_fft (= i*1024 + 1024)
    # min(n_fft, n - sample) --> n_fft (=1024)
    # max(0, min(n_fft, n - sample) -->   n_fft (=1024)
# return x

x.shape ## (326400,)

1024 1024 1024 1272
326400 (326400,)


(326400,)

In [96]:
## 의미 파악은 포기했다. 
window_sum = window_sumsquare(
    window,
    magnitude.size(-1),
    hop_length=hop_length,
    win_length=win_length,
    n_fft=filter_length,
    dtype=np.float32,
)
window_sum.shape # (326400,)

(326400,)

In [97]:
# tiny(11) #### 1.1754944e-38 ???????????????????
print(window_sum[:3])       # [0.0000000e+00 8.8592494e-11 1.4174533e-09]
print(tiny(window_sum[:3])) # 1.1754944e-38 

[0.0000000e+00 8.8592494e-11 1.4174533e-09]
1.1754944e-38


In [98]:
# np.where(np.arange(0, 12) > 9)

In [99]:
# remove modulation effects
approx_nonzero_indices = torch.from_numpy(np.where(window_sum > tiny(window_sum))[0])
## np.where: : 조건 만족하는 위치 인덱스 찾기
print(approx_nonzero_indices[:5])
approx_nonzero_indices.shape

tensor([1, 2, 3, 4, 5])


torch.Size([326399])

In [100]:
window_sum = torch.autograd.Variable(
    torch.from_numpy(window_sum), requires_grad=False) # (326400,)
window_sum.shape

torch.Size([326400])

In [102]:
# window_sum = window_sum.cuda() if magnitude.is_cuda else window_sum
# window_sum : [[326400]
# approx_nonzero_indices : [326399]
# inverse_transform: [1, 1, 326399]
inverse_transform[:, :, approx_nonzero_indices] /= window_sum[approx_nonzero_indices]
inverse_transform[:5]
# inverse_transform: [1, 1, 326399]

tensor([[[ 0.0000e+00,  2.5059e-15,  3.3675e-14,  ..., -1.2417e-11,
          -2.5097e-12, -1.5316e-13]]])

In [103]:
# scale by hop ratio
inverse_transform *= float(filter_length) / hop_length
# inverse_transform: [1, 1, 326399]
inverse_transform[:5]

tensor([[[ 0.0000e+00,  1.0023e-14,  1.3470e-13,  ..., -4.9668e-11,
          -1.0039e-11, -6.1266e-13]]])

In [105]:
inverse_transform = inverse_transform[:, :, int(filter_length / 2) :]
print(inverse_transform.shape)
inverse_transform[:5]

torch.Size([1, 1, 325888])


tensor([[[ 3.6049e-03,  4.1520e-03,  3.6090e-03,  ..., -4.9668e-11,
          -1.0039e-11, -6.1266e-13]]])

In [107]:
inverse_transform = inverse_transform[:, :, : -int(filter_length / 2) :]
print(inverse_transform.shape)
inverse_transform[:5]

torch.Size([1, 1, 325376])


tensor([[[ 0.0036,  0.0042,  0.0036,  ..., -0.0052, -0.0052, -0.0048]]])

In [109]:
325888 - 512

325376

In [110]:
audio.shape

(325458,)